**Project: 150.Lead-Data-Engineer-CodeSignal-Sprint**

**Day 2**: Sliding Window (Variable Size)

### LeetCode 76. Minimum Window Substring

Given two strings `s` and `t` of lengths `m` and `n` respectively, return *the minimum window substring of* `s` *such that every character in* `t` *(including duplicates) is included in the window*.
If there is no such substring, return the empty string `""`.

The testcases will be generated such that the answer is unique.

**Example 1:**
```
Input: s = "ADOBECODEBANC", t = "ABC"
Output: "BANC"
Explanation: The minimum window substring "BANC" includes 'A', 'B', and 'C' from string t.
```

**Example 2:**
```
Input: s = "a", t = "a"
Output: "a"
```

**Example 3:**
```
Input: s = "a", t = "aa"
Output: ""
Explanation: Both 'a's from t must be included in the window.
Since the largest window of s only has one 'a', return empty string.
```

**Constraints:**
- m == s.length
- n == t.length
- 1 <= m, n <= 10^5
- s and t consist of uppercase and lowercase English letters.


In [117]:
# ------------------------------------------------------------------
# Test Harness
# ------------------------------------------------------------------
def run_tests(func):
    test_cases = [
        # --- LeetCode examples ---
        ("Example 1", "ADOBECODEBANC", "ABC", "BANC"),
        ("Example 2", "a", "a", "a"),
        ("Example 3", "a", "aa", ""),

        # --- Edge cases ---
        ("t longer than s", "a", "ab", ""),
        ("Exact match sequence", "abc", "abc", "abc"),
        ("Reversed sequence", "cba", "abc", "cba"),
        ("Entire string needed", "abcde", "abcde", "abcde"),
        ("No match found", "abcdef", "z", ""),
        ("Duplicate characters in t", "aaabbb", "ab", "ab"),
        ("Many duplicates", "aaabbbccc", "abc", "abbbc"),
        ("Substruing at end", "xxxyyyzzzabc", "abc", "abc"),
        ("Substring at start", "abcxxxyyyzzz", "abc", "abc"),
        ("Middle match", "xxabcxx", "abc", "abc"),
        ("Mixed case", "aAa", "aa", "aAa"),
        # Classic trap: multiple overlapping candidates — must shrink aggressively to find smallest
("Overlapping candidates", "bba", "ab", "ba"),               # should return "ba" (length 2), not "bba" (length 3)

# t has many duplicates, s has just enough but spread out
("Many required duplicates", "aabaabaa", "aaaa", "aabaa"),   # correct = "aabaa" (length 5)

# Minimal window is in the middle, surrounded by extra chars of the same type
("Extra chars around", "azzzabczzz", "abc", "abc"),           # should return "abc" (not longer)

# t requires more of one char than appears consecutively
("Insufficient consecutive", "aaabbbcccddd", "aabbcc", "aabbbcc"),  # correct = "aaabbbcc" (length 8)

# Window must include later occurrences to satisfy count
("Later occurrence needed", "ADOBECODEBANCA", "AABC", "BANCA"),     # correct = "BANCA" (includes second A)

# Empty t (should return "")
("Empty t", "anything", "", ""),

# t with one char, s with multiple
("Single char many times", "aaaba", "a", "a")
    ]

    print(f"Running tests for: {func.__name__}\n")
    passed = 0
    for desc, s, t, expected in test_cases:
        result = func(s, t)
        if result == expected:
            print(f"✅ {desc}: Passed")
            passed += 1
        else:
            print(f"❌ {desc}: Failed")
            print(f"   Input: s={s}, t={t}")
            print(f"   Expected: {expected}")
            print(f"   Actual:   {result}")
        print("-" * 30)
    print(f"\n{passed}/{len(test_cases)} passed\n")

run_tests(minWindow)

# Seans Choice Solution

![Sean's Solution](attachments/1.png)

In [121]:
from collections import Counter
import sys
"""
    LeetCode 76. Minimum Window Substring
    
    Given strings s and t, return the minimum window in s which will contain all the characters in t.
    
    Solution Approach: -
        OOP. Use Object Window and Object Result
        Class Result it keeps Track of the minimum possible solution
        Class Window
            Keeps Track off
                Counter of the target string and main string to search in
                keeps track of its length and its starting index and ending index
                systematically changes left and right pointer as it expands or shrinks  while updating the Counter of the have .. need is fixed
                Behaviour: -
                    A) systematically maintains window length
                    B) updates the have dictionary with adds to right and removes at left
                    C) calculates matches between the need and have 
                    D) it can process special for cases of "" or a single letter to search for
                    E) It shrinks and expands maintaining that it never exceed the string limits. 
                        While changing its size and position, it updates let and right pointer,
                            the winfow length, the have Counter
    
"""
        
class Window:
    def __init__(self, s: str, t: str):
        self.s = s
        self.t = t
        self.sLength = len(s)
        self.left = 0
        self.right = 0
        self.length = 1
        self.need = Counter(t)
        self.have = self._getEmptyHave()
        self._updateHaveAdd(s[0])

    def _getEmptyHave(self)->Counter: #Inner helper method
        return  Counter({key: 0 for key in self.need})

    def _updateLength(self): #Inner helper method
        self.length = self.right - self.left + 1
    
    def _updateHaveAdd(self, ch: str)->None: #Inner helper method
        if ch in self.have:
            self.have[ch] += 1

    def _updateHaveRemove(self, ch: str)->None: #Inner helper method
        if ch in self.have:
            self.have[ch] -= 1

    def matches(self)->bool:
        """
         Calculates matches for have has at least the same combination as need
        """
        for ch in self.need:
            if self.have[ch] < self.need[ch]:
                return False
        return True

    def wind(self)->str:
        return self.s[self.left:self.right+1]

    def expand(self)->bool:
        """
            expanding; maintain pointer, length and the have
        """
        if self.right < self.sLength - 1:
            self.right += 1
            self._updateLength()
            self._updateHaveAdd(self.s[self.right])
            return True
        return False
        """
            shrink ; maintain pointer, length and the have
        """
    def shrink(self)->bool:
        if self.left < self.right:
            self._updateHaveRemove(self.s[self.left])
            self.left += 1
            self._updateLength()
            return True
        return False    

    def process_special(s: str, t: str)->str:
        """
            returns empty string if both s, t are not 
            if length of t is 1 ... it will check if t is in s and return "t" if it exists
            returns None if it is not special case
        """
        if len(t) > len(s):
            return ""   
        if not t:  
            return ""
        if not s:  
            return ""
        if len(t) == 1:
            if t in s:
                return t
            else:
                return None
        return None

class Result:
    def __init__(self):
        self.length = float('inf')
        self.s = ""

    def update(self, s: str):
        if len(s) < self.length:
            self.length = len(s)
            self.s = s

def  minWindow(s: str, t: str) -> str:
    special = Window.process_special(s, t)
    if special is not None:
        return special

    w: Window = Window(s, t)

    res: Result = Result()
    while w.expand():
        if w.matches():
            res.update(w.wind())
            while(w.shrink()):
                if w.matches():
                    res.update(w.wind())
                else:
                    break
    return res.s
print("ADOBECODEBANC" , "ABC", "Output ", minWindow ("ADOBECODEBANC" , "ABC"))
print("cabwefgewcwaef" , "cae", "Output ", minWindow ("cabwefgewcwaef" , "cae"))
run_tests(minWindow)

In [4]:
from collections import Counter


def minWindow_BruteForce(s: str, t: str) -> str:
    if not t or not s:
        return ""

    t_count = Counter(t)
    best = ""

    for i in range(len(s)):
        for j in range(i + len(t), len(s) + 1):  # window must be at least len(t)
            window = s[i:j]
            window_count = Counter(window)

            # Check: does window have enough of every char in t?
            if all(window_count[c] >= t_count[c] for c in t_count):
                if best == "" or len(window) < len(best):
                    best = window
                break  # expanding further from same i won't be shorter

    return best
run_tests(minWindow_BruteForce)

In [6]:
from collections import Counter


def minWindow_SlidingCounter(s: str, t: str) -> str:
    if not t or not s:
        return ""

    t_count = Counter(t)
    window_count = Counter()

    best_start, best_len = 0, float('inf')
    left = 0

    for right in range(len(s)):
        # Expand: add right char
        window_count[s[right]] += 1

        # Contract: while window is valid, try shrinking from left
        while all(window_count[c] >= t_count[c] for c in t_count):
            window_size = right - left + 1
            if window_size < best_len:
                best_len = window_size
                best_start = left

            window_count[s[left]] -= 1
            left += 1

    return s[best_start:best_start + best_len] if best_len != float('inf') else ""
run_tests(minWindow_SlidingCounter)

In [8]:
from collections import Counter


def minWindow_HaveNeed(s: str, t: str) -> str:
    if not t or not s:
        return ""

    t_count = Counter(t)
    need = len(t_count)       # number of unique chars we need
    have = 0                  # how many unique chars currently satisfied
    window_count = {}

    best_start, best_len = 0, float('inf')
    left = 0

    for right in range(len(s)):
        char = s[right]
        window_count[char] = window_count.get(char, 0) + 1

        # Did adding this char satisfy a requirement?
        if char in t_count and window_count[char] == t_count[char]:
            have += 1

        # Contract while we have a valid window
        while have == need:
            window_size = right - left + 1
            if window_size < best_len:
                best_len = window_size
                best_start = left

            # Shrink from left
            left_char = s[left]
            window_count[left_char] -= 1
            if left_char in t_count and window_count[left_char] < t_count[left_char]:
                have -= 1
            left += 1

    return s[best_start:best_start + best_len] if best_len != float('inf') else ""

run_tests(minWindow_SlidingCounter)

In [122]:
from collections import Counter

class Window:
    def __init__(self, s: str, t: str):
        self.s = s
        self.t = t
        self.sLength = len(s)
        self.left = 0
        self.right = 0
        self.length = 1
        self.need = Counter(t)
        self.have = self._getEmptyHave()
        self.satisfied = 0
        self.need_count = len(self.need)
        self._updateHaveAdd(s[0])

    def _getEmptyHave(self) -> Counter:
        return Counter({key: 0 for key in self.need})

    def _updateLength(self):
        self.length = self.right - self.left + 1

    def _updateHaveAdd(self, ch: str) -> None:
        if ch in self.have:
            self.have[ch] += 1
            if self.have[ch] == self.need[ch]:
                self.satisfied += 1

    def _updateHaveRemove(self, ch: str) -> None:
        if ch in self.have:
            if self.have[ch] == self.need[ch]:
                self.satisfied -= 1
            self.have[ch] -= 1

    def matches(self) -> bool:
        return self.satisfied == self.need_count

    def wind(self) -> str:
        return self.s[self.left:self.right + 1]

    def expand(self) -> bool:
        if self.right < self.sLength - 1:
            self.right += 1
            self._updateLength()
            self._updateHaveAdd(self.s[self.right])
            return True
        return False

    def shrink(self) -> bool:
        if self.left < self.right:
            self._updateHaveRemove(self.s[self.left])
            self.left += 1
            self._updateLength()
            return True
        return False

    def process_special(s: str, t: str) -> str:
        if len(t) > len(s):
            return ""
        if not t:
            return ""
        if not s:
            return ""
        if len(t) == 1:
            if t in s:
                return t
            else:
                return None
        return None


class Result:
    def __init__(self):
        self.length = float('inf')
        self.s = ""

    def update(self, s: str):
        if len(s) < self.length:
            self.length = len(s)
            self.s = s


def minWindow(s: str, t: str) -> str:
    special = Window.process_special(s, t)
    if special is not None:
        return special

    w = Window(s, t)
    res = Result()

    while w.expand():
        if w.matches():
            res.update(w.wind())
            while w.shrink():
                if w.matches():
                    res.update(w.wind())
                else:
                    break
    return res.s







print("", "a", "Expected: '', Got:", minWindow("", "a"))
print("a", "aa", "Expected: '', Got:", minWindow("a", "aa"))
print("a", "b", "Expected: '', Got:", minWindow("a", "b"))
print("abc", "b", "Expected: b, Got:", minWindow("abc", "b"))
print("abc", "ac", "Expected: abc, Got:", minWindow("abc", "ac"))
print("aa", "aa", "Expected: aa, Got:", minWindow("aa", "aa"))
print("ADOBECODEBANC", "ABC", "Expected: BANC, Got:", minWindow("ADOBECODEBANC", "ABC"))
print("a", "a", "Expected: a, Got:", minWindow("a", "a"))
print("a", "a", "Expected: a, Got:", minWindow("a", "a"))
# Match at position 0 with shorter window than full string
print("ab", "a", "Expected: a, Got:", minWindow("ab", "a"))

# Match is the very first character
print("aXYZ", "a", "Expected: a, Got:", minWindow("aXYZ", "a"))

# Best match is at the start, longer t
print("abcXXXcba", "abc", "Expected: abc, Got:", minWindow("abcXXXcba", "abc"))

# Entire string is the answer and match exists from first expansion
print("ba", "ab", "Expected: ba, Got:", minWindow("ba", "ab"))

# Single char s and t that match (without process_special saving it)
print("ab", "ab", "Expected: ab, Got:", minWindow("ab", "ab"))

 a Expected: '', Got: 
a aa Expected: '', Got: 
a b Expected: '', Got: 
abc b Expected: b, Got: b
abc ac Expected: abc, Got: abc
aa aa Expected: aa, Got: aa
ADOBECODEBANC ABC Expected: BANC, Got: BANC
a a Expected: a, Got: a
a a Expected: a, Got: a
ab a Expected: a, Got: a
aXYZ a Expected: a, Got: a
abcXXXcba abc Expected: abc, Got: abc
ba ab Expected: ba, Got: ba
ab ab Expected: ab, Got: ab


In [ ]:
Broken / Wrong (2 clear failures)

ADOBECODEBAN ABC
Expected: BANC
Got: ADOBEC
→ Wrong — returns a longer prefix instead of the minimal "BANC" at the end.
aabbcc abc
Expected: aabbcc (whole string)
Got: abbc
→ Wrong — returns a substring that misses one 'a' (invalid), should be the full "aabbcc".